Import Required Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Conv2D, UpSampling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import MeanIoU


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Debugging to check the contents of the directories
def debug_directory(path):
    for root, dirs, files in os.walk(path):
        print(f"Root: {root}")
        print(f"Dirs: {dirs}")
        print(f"Files: {files}")

debug_directory('/content/drive/MyDrive/ProstateX_dataset')
debug_directory('/content/drive/MyDrive/ProstateX_mask')


Root: /content/drive/MyDrive/ProstateX_dataset
Dirs: ['images']
Files: []
Root: /content/drive/MyDrive/ProstateX_dataset/images
Dirs: []
Files: ['ProstateX-0150_03-15-2012-NA-MR prostaat kanker detectie WDSmc MCAPRODETW-08825_4.000000-t2tsetra-41386_1-19.png', 'ProstateX-0154_03-21-2012-NA-MR prostaat kanker detectie WDSmc MCAPRODETW-88301_4.000000-t2tsetra-89111_1-11.png', 'ProstateX-0150_03-15-2012-NA-MR prostaat kanker detectie WDSmc MCAPRODETW-08825_4.000000-t2tsetra-41386_1-03.png', 'ProstateX-0150_03-15-2012-NA-MR prostaat kanker detectie WDSmc MCAPRODETW-08825_4.000000-t2tsetra-41386_1-17.png', 'ProstateX-0146_03-08-2012-NA-MR prostaat kanker detectie WDSmc MCAPRODETW-87656_4.000000-t2tsetra-76067_1-20.png', 'ProstateX-0154_03-21-2012-NA-MR prostaat kanker detectie WDSmc MCAPRODETW-88301_4.000000-t2tsetra-89111_1-08.png', 'ProstateX-0150_03-15-2012-NA-MR prostaat kanker detectie WDSmc MCAPRODETW-08825_4.000000-t2tsetra-41386_1-05.png', 'ProstateX-0154_03-21-2012-NA-MR prostaat k

Define Dice Coefficient Metric

In [ ]:
def dice_coefficient(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)


Load the VGG16 Model

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Build the SegNet Model (Decoder)

In [ ]:


# Encoder from VGG16
encoder_output = base_model.output

# Existing decoder layers
decoder = UpSampling2D((2, 2))(encoder_output)  # Upsample to 28x28
decoder = Conv2D(512, (3, 3), padding='same', activation='relu')(decoder)

decoder = UpSampling2D((2, 2))(decoder)  # Upsample to 56x56
decoder = Conv2D(256, (3, 3), padding='same', activation='relu')(decoder)

decoder = UpSampling2D((2, 2))(decoder)  # Upsample to 112x112
decoder = Conv2D(128, (3, 3), padding='same', activation='relu')(decoder)

decoder = UpSampling2D((2, 2))(decoder)  # New layer to upsample to 224x224
decoder = Conv2D(64, (3, 3), padding='same', activation='relu')(decoder)

decoder = UpSampling2D((2, 2))(decoder)  # Upsample to 224x224
decoder = Conv2D(64, (3, 3), padding='same', activation='relu')(decoder)

# Final output layer to produce 1 channel mask
decoder = Conv2D(1, (1, 1), padding='same', activation='sigmoid')(decoder)  # Output shape: (224, 224, 1)

# Combine Encoder and Decoder into a Model
model = Model(inputs=base_model.input, outputs=decoder)


In [ ]:
# Check the model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d (UpSampling2D)         │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 18,660,225 (71.18 MB)

 Trainable params: 3,945,537 (15.05 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

Combine Encoder and Decoder into a Model

In [ ]:
model = Model(inputs=base_model.input, outputs=decoder)


Prepare the Data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ImageDataGenerator for data augmentation
"""train_datagen = ImageDataGenerator(rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   rescale=1./255)"""
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/ProstateX_dataset',
    target_size=(224, 224),
    batch_size=16,
    class_mode=None,
    color_mode='rgb')

mask_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/ProstateX_mask',
    target_size=(224, 224),
    batch_size=16,
    class_mode=None,
    color_mode='grayscale')

# Combine image and mask generators
def combined_generator(image_gen, mask_gen):
    while True:
        image_batch = next(image_gen)
        mask_batch = next(mask_gen)
        # Ensure that the shapes of images and masks match
        assert image_batch.shape[1:] == (224, 224, 3), f"Image shape mismatch: {image_batch.shape}"
        assert mask_batch.shape[1:] == (224, 224, 1), f"Mask shape mismatch: {mask_batch.shape}"
        yield image_batch, mask_batch


train_gen = combined_generator(train_generator, mask_generator)


for images, masks in train_gen:
    print(f'Images shape: {images.shape}')
    print(f'Masks shape: {masks.shape}')
    break  # Only check one batch


Found 2020 images belonging to 1 classes.
Found 2020 images belonging to 1 classes.
Images shape: (16, 224, 224, 3)
Masks shape: (16, 224, 224, 1)


Compile the Model

In [ ]:
#
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=BinaryCrossentropy(),
              metrics=[dice_coefficient, MeanIoU(num_classes=2)],
              run_eagerly=True)




In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


Train the model

In [ ]:
# Train the model using the custom generator
model.fit(train_gen,
          steps_per_epoch=len(train_generator),
          epochs=10)


Epoch 1/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1239s 10s/step - dice_coefficient: 0.0263 - loss: 0.1091 - mean_io_u: 0.4955
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 65s 507ms/step - dice_coefficient: 0.0924 - loss: 0.0303 - mean_io_u: 0.4957
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 67s 526ms/step - dice_coefficient: 0.0888 - loss: 0.0307 - mean_io_u: 0.4958
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 67s 531ms/step - dice_coefficient: 0.1066 - loss: 0.0292 - mean_io_u: 0.4957
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 68s 537ms/step - dice_coefficient: 0.1021 - loss: 0.0292 - mean_io_u: 0.4957
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 67s 526ms/step - dice_coefficient: 0.1099 - loss: 0.0286 - mean_io_u: 0.4957
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 67s 531ms/step - dice_coefficient: 0.1054 - loss: 0.0291 - mean_io_u: 0.4957
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 66s 518ms/step - dice_coefficient: 0.1070 - loss: 0.0286 - mean_io_u: 0.4958
Epoch 9/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 67s 528ms/step - dice_co